In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/martin/Documents/git/GNN/torch_geometric_examples/") 
# custom dataset
from division_v import DivisionV
from division import Division

import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import r2_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv

In [2]:
# constants

PATH = '/home/martin/graphs/division'

In [3]:

train_dataset   = DivisionV(PATH, split='train')
val_dataset     = DivisionV(PATH, split='val')
test_dataset    = DivisionV(PATH, split='test')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

#train_dataset[0].num_classes



Processing...


Creating splits: train: 0:80, valid: 81:91, test: 92:100


Done!


In [4]:
len(train_dataset[0])

2

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(train_dataset.num_features, 256, heads=4, edge_dim=1)
        self.lin1 = torch.nn.Linear(train_dataset.num_features, 4 * 256)
        self.conv2 = GATConv(4 * 256, 256, heads=4, edge_dim=1)
        self.lin2 = torch.nn.Linear(4 * 256, 4 * 256)
        self.conv3 = GATConv(4 * 256, 1, heads=6,
                             concat=False, edge_dim=1)
        self.lin3 = torch.nn.Linear(4 * 256, 1)

    def forward(self, x, edge_index, edge_attr):
        x = F.relu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.relu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x

In [6]:
from torchsummary import summary

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [8]:
su = summary(model)

Layer (type:depth-idx)                   Param #
├─GATConv: 1-1                           --
|    └─Linear: 2-1                       1,024
|    └─Linear: 2-2                       1,024
├─Linear: 1-2                            2,048
├─GATConv: 1-3                           --
|    └─Linear: 2-3                       1,048,576
|    └─Linear: 2-4                       1,024
├─Linear: 1-4                            1,049,600
├─GATConv: 1-5                           --
|    └─Linear: 2-5                       6,144
|    └─Linear: 2-6                       6
├─Linear: 1-6                            1,025
Total params: 2,110,471
Trainable params: 2,110,471
Non-trainable params: 0


In [9]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:

        data = data[0]
        
        data = data.to(device)
        optimizer.zero_grad()
        rt = model(data.x, data.edge_index, data.edge_attr)
        loss = loss_op(rt, data.y)
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)

In [20]:


for epoch in range(1, 1001):
    loss = train()
    # val_f1 = test(val_loader)
    # test_f1 = test(test_loader)
    # print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_f1:.4f}, '
    #       f'Test: {test_f1:.4f}')

In [21]:
ID = 8
DS = 0
model.eval()
out = model(test_dataset[DS][0].x.to(device), test_dataset[DS][0].edge_index.to(device), test_dataset[DS][0].edge_attr)
print(test_dataset[DS][0].x.numpy()[ID])
out.cpu().detach().numpy()[ID][0]


[0.60001963]


0.058615923

In [22]:
loss

5.214478164816683e-07